# Assignment 3

### <span style="color:chocolate"> Submission requirements </span>

Your work will not be graded if your notebook doesn't include output. In other words, <span style="color:red"> make sure to rerun your notebook before submitting to Gradescope </span> (Note: if you are using Google Colab: go to Edit > Notebook Settings  and uncheck Omit code cell output when saving this notebook, otherwise the output is not printed).

Additional points may be deducted if these requirements are not met:

    
* Comment your code;
* Each graph should have a title, labels for each axis, and (if needed) a legend. Each graph should be understandable on its own;
* Try and minimize the use of the global namespace (meaning, keep things inside functions).
---

### Import libraries

In [39]:
import numpy as np
import pandas as pd
import seaborn as sns  # for nicer plots
sns.set(style="darkgrid")  # default style
from sklearn.model_selection import train_test_split
import tensorflow as tf
from matplotlib import pyplot as plt
import keras_tuner as kt
from keras_tuner import HyperParameters

This lab continues our study of linear regression. You'll train your first models with Tensorflow, using a real dataset to predict car prices from their features. Note that Tensorflow is a rapidly changing library. This means you'll often see warnings about deprecations. You can ignore the warnings in our labs.

---
### Step 1: Data ingestion

You'll use the [Automobile Data Set](https://archive.ics.uci.edu/ml/datasets/automobile)  from 1985 Ward's Automotive Yearbook that is part of the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets).

This dataset has been pre-downloaded and uploaded to bCourses alongside this notebook for your convenience.

In [40]:
#car_data_init = pd.read_csv("./data/Automobile_data/automobile_data.txt")
# Slightly simplied data directory
car_data_init = pd.read_csv("./data/automobile_data.txt", keep_default_na=False)

# Display top five rows
print('Shape of data:', car_data_init.shape)
car_data_init.head(30)

Shape of data: (205, 26)


,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,...,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price,symboling
0,NaN,alfa-romero,gas,std,2,convertible,rwd,front,88.6,168.8,...,mpfi,3.47,2.68,9.00,111,5000,21,27,13495,3
1,NaN,alfa-romero,gas,std,2,convertible,rwd,front,88.6,168.8,...,mpfi,3.47,2.68,9.00,111,5000,21,27,16500,3
2,NaN,alfa-romero,gas,std,2,hatchback,rwd,front,94.5,171.2,...,mpfi,2.68,3.47,9.00,154,5000,19,26,16500,1
3,164,audi,gas,std,4,sedan,fwd,front,99.8,176.6,...,mpfi,3.19,3.4,10.00,102,5500,24,30,13950,2
4,164,audi,gas,std,4,sedan,4wd,front,99.4,176.6,...,mpfi,3.19,3.4,8.00,115,5500,18,22,17450,2
5,NaN,audi,gas,std,2,sedan,fwd,front,99.8,177.3,...,mpfi,3.19,3.4,8.50,110,5500,19,25,15250,2
6,158,audi,gas,std,4,sedan,fwd,front,105.8,192.7,...,mpfi,3.19,3.4,8.50,110,5500,19,25,17710,1
7,NaN,audi,gas,std,4,wagon,fwd,front,105.8,192.7,...,mpfi,3.19,3.4,8.50,110,5500,19,25,18920,1
8,158,audi,gas,turbo,4,sedan,fwd,front,105.8,192.7,...,mpfi,3.13,3.4,8.30,140,5500,17,20,23875,1
9,NaN,audi,gas,turbo,2,hatchback,4wd,front,99.5,178.2,...,mpfi,3.13,3.4,7.00,160,5500,16,22,NaN,0


---
### Step 2: Data preprocessing

This step is essential for preparing the data in a format that is suitable for ML algorithms. It helps ensure data quality and improvements in model performance.

### <span style="color:chocolate">Exercise 1:</span> Column selection (5 points)

To keep things simple, you will:

1. Retain only the following columns: ['horsepower', 'peak-rpm', 'city-mpg', 'highway-mpg', 'price']. Name the new dataframe *car_data*.
2. Display the data type of each column;
3. Convert the data type of each columns to numeric. Coerce missing values to NaN. Hint: use <span style="color:chocolate">pd.to_numeric()</span> method;
4. Display the data type of each column after the transformation performed at point 3.


In [41]:
# YOUR CODE HERE

# Task 1 
cols_to_keep = ['horsepower', 'peak-rpm', 'city-mpg', 'highway-mpg', 'price']
car_data = car_data_init[cols_to_keep]
# Task 2
print("2: Column data types along with missing values:\n")
print(car_data.info())
print(car_data.dtypes)

# Task 3
car_data = car_data.apply(pd.to_numeric, errors='coerce')
car_data.dtypes

2: Column data types along with missing values:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   horsepower   205 non-null    object
 1   peak-rpm     205 non-null    object
 2   city-mpg     205 non-null    int64 
 3   highway-mpg  205 non-null    int64 
 4   price        205 non-null    object
dtypes: int64(2), object(3)
memory usage: 8.1+ KB
None
horsepower     object
peak-rpm       object
city-mpg        int64
highway-mpg     int64
price          object
dtype: object


horsepower     float64
peak-rpm       float64
city-mpg         int64
highway-mpg      int64
price          float64
dtype: object

### <span style="color:chocolate">Exercise 2:</span> Example (row) selection (5 points)

To keep things simple again, you will:

1. Print the shape of the car_data;

2. Remove examples (rows) that have missing value(s). Note that in doing so, you will overwrite the car_data dataset. You should end up with 199 examples after this cleaning.

3. Print the shape of the car_data again.

It's important to acknowledge that there are multiple approaches to handling missing features, and simply discarding examples with any missing feature, though straightforward, may not be the most optimal solution. However, for the sake of simplicity, you will implement this strategy in this assignment.

In [42]:
# YOUR CODE HERE

# Task 1
print("Shape of car_data BEFORE removing NAs:", car_data.shape)
car_data = car_data.dropna().reset_index(drop=True)

print("car_data shape after removing NAs:", car_data.shape)

Shape of car_data BEFORE removing NAs: (205, 5)
car_data shape after removing NAs: (199, 5)


### <span style="color:chocolate">Exercise 3:</span> Data shuffling (10 points)

Since you'll be using Batch Gradient Descent (BGD) for training, it is important that **each batch is a random sample of the data** so that the gradient computed is representative. Note that the original data (above) appears sorted by *make* in alphabetic order.

Using NumPy and Pandas methods:

1. Create a list of indices corresponding to the rows in the car_data dataset. Call this list *indices*. Print this list;

2. Shuffle *indices* using the <span style="color:chocolate">np.random.permutation()</span> method. Call the resulting array *shuffled_indices*. Print this array;
    
3. Use the method <span style="color:chocolate">dataframe.reindex()</span> to change the ordering of the car_data dataset based on the order in the *shuffled_indices* array. Note that in doing so, you will overwrite the original dataset. Print the top 5 rows.

In [43]:
np.random.seed(0)
# YOUR CODE HERE

# Assumption: I assume that by list, we are not limited to python list. A numpy list will do too.
indices = np.arange(car_data.shape[0])
indices.shape
shuffled_indices = np.random.permutation(indices)
type(shuffled_indices)

print(car_data.head(10))
car_data = car_data.reindex(shuffled_indices)
print(car_data.head(5))

   horsepower  peak-rpm  city-mpg  highway-mpg    price
0       111.0    5000.0        21           27  13495.0
1       111.0    5000.0        21           27  16500.0
2       154.0    5000.0        19           26  16500.0
3       102.0    5500.0        24           30  13950.0
4       115.0    5500.0        18           22  17450.0
5       110.0    5500.0        19           25  15250.0
6       110.0    5500.0        19           25  17710.0
7       110.0    5500.0        19           25  18920.0
8       140.0    5500.0        17           20  23875.0
9       101.0    5800.0        23           29  16430.0
     horsepower  peak-rpm  city-mpg  highway-mpg    price
18         70.0    5400.0        38           43   6295.0
169        92.0    4200.0        27           32   9988.0
106        97.0    5000.0        19           24  12440.0
92         69.0    5200.0        31           37   7799.0
176        52.0    4800.0        37           46   7775.0


### <span style="color:chocolate">Exercise 4:</span> Define outcome and features (5 points)

Create two dataframes as follows:

1. The first dataframe contains our outcome of interest: ['price']. Note, this is what we are aiming to predict. Name this dataframe Y. Print shape of Y.
2. The second dataframe contains our features of interest: ['horsepower', 'peak-rpm', 'city-mpg', 'highway-mpg']. Name this dataframe X. Print shape of X.


In [44]:
# YOUR CODE HERE

Y = car_data.price.values
feature_cols = [col for col in cols_to_keep if col != 'price']
feature_cols
X = car_data[feature_cols].values
print("Y shape:", Y.shape)
print("X shape:", X.shape)

Y shape: (199,)
X shape: (199, 4)


### <span style="color:chocolate">Exercise 5:</span> Data splits (10 points)

Using the <span style="color:chocolate">train_test_split()</span> method available in scikit-learn:
1. Partition the (X, Y) data into training, validation, and test sets using a splitting rule of [60%, 20%, 20%], with a random state set to 1234. Name the resulting dataframes as follows: X_train, X_val, X_test, Y_train, Y_val, Y_test. Hint: To create these three partitions you will utilize the train_test_split() method twice (all the other arguments of the method are set to default values.). You should obtain [119, 40, 40] examples for training, validation, and test, respectively.
2. Print the shape of each dataframe.

Note: The validation set is crucial for evaluating different hyperparameter configurations and selecting those that yield optimal model performance. This approach avoids utilizing the test dataset during model training, as it is assumed to be "unknown" at that stage.

In [57]:
# YOUR CODE HERE
X_train_val, X_test, Y_train_val, Y_test = train_test_split(X, Y, test_size=0.2)
X_train, X_val, Y_train, Y_val = train_test_split(X_train_val, Y_train_val, test_size=0.25) 
print("X_train shape:", X_train.shape)
print("X_val shape:", X_val.shape)
print("X_test shape", X_test.shape)

print(50*"*")

print("Y_train shape:", Y_train.shape)
print("Y_val", Y_val.shape)
print("Y_test", Y_test.shape)


X_train shape: (119, 4)
X_val shape: (40, 4)
X_test shape (40, 4)
**************************************************
Y_train shape: (119,)
Y_val (40,)
Y_test (40,)


### <span style="color:chocolate">Exercise 6:</span> Data standardization (10 points)

With this concept in mind, complete the following tasks:

1. Output the quantile values (0.25, 0.5, 0.75, 0.95) for all features in the X_train dataset. Are these values uniformly scaled across features?

2. Standardize all features in X_train, X_val, and X_test. Label the resulting dataframes as X_train_std, X_val_std, and X_test_std, respectively. Hint: standardize the validation and test data using the mean and standard deviation computed from the training data. Why?

3. Similar to point 2. but now standardize the outcome variable. Label the resulting dataframes as Y_train_std, Y_val_std, and Y_test_std.

In [99]:
# YOUR CODE HERE
X_train
X_train
np.percentile(a=X_train, 
              q=[25, 50, 75, 95],
              axis=0)
def standardize(inputs):
    """  
    Return standardized array 
    Args:
        inputs: a numpy array
    """
    return (inputs - np.mean(a=inputs,
        axis=0, keepdims=True))/np.std(a=inputs,
       axis=0, keepdims=True)

X_train_std = standardize(X_train) 
X_val = standardize(X_val) 
X_test = standardize(X_test)

Y_train_std = standardize(Y_train) 
Y_val = standardize(Y_val) 
Y_test = standardize(Y_test)



---
### Step 3: Exploratory data analysis (EDA)

EDA plays a very important role in ML. The goal here is to develop a good understanding of our training dataset, identify any data quality issues, understand patterns and relationships, which in turn, aids in subsequent modeling and interpretations.

### <span style="color:chocolate">Exercise 7:</span> Scatterplot matrix (10 points)

In this exercise you will use some simple yet useful techniques to visualize the distribution of the data. 

Let's start with:

1. A scatterplot matrix to visualize the pair-wise correlations between different features and outcome in the (X_train_std, Y_train_std) data. You will use the <span style="color:chocolate">sns.pairplot()</span> method from the seaborn library imported at the top of the notebook;
2. Is any of the variables in the data normally distributed? Is it necessary for the explanatory or target variable to be normally distributed in order to train a ML model?

In [112]:
# YOUR CODE HERE
#sns.pairplot(pd.DataFrame(X_train, Y_train), diag_kind='hist')
print(car_data)
pd.DataFrame(X_train, Y_train)


     horsepower  peak-rpm  city-mpg  highway-mpg    price
18         70.0    5400.0        38           43   6295.0
169        92.0    4200.0        27           32   9988.0
106        97.0    5000.0        19           24  12440.0
92         69.0    5200.0        31           37   7799.0
176        52.0    4800.0        37           46   7775.0
..          ...       ...       ...          ...      ...
67        123.0    4350.0        22           25  31600.0
192       162.0    5100.0        17           22  18420.0
117        68.0    5500.0        31           38   6229.0
47         68.0    5000.0        30           31   5195.0
172       161.0    5200.0        20           24  16558.0

[199 rows x 5 columns]


,0,1,2,3
11595.0,90.0,5500.0,24.0,29.0
8845.0,84.0,4800.0,26.0,32.0
10198.0,94.0,5200.0,25.0,31.0
13499.0,152.0,5200.0,19.0,25.0
11845.0,101.0,6000.0,17.0,23.0
...,...,...,...,...
9258.0,70.0,4800.0,28.0,34.0
10595.0,84.0,4800.0,26.0,32.0
6855.0,76.0,6000.0,31.0,38.0
6229.0,68.0,5500.0,31.0,38.0


### <span style="color:chocolate">Exercise 8:</span> Correlation matrix (10 points)

In this exercise you will:

1. Plot a correlation matrix in the form of a heatmap to visualize the linear relationships between different features and outcome in the (X_train_std, Y_train_std) data. Hint: this example here is very useful: https://seaborn.pydata.org/examples/many_pairwise_correlations.html
    
2. Answer the following questions: 
 - Which two features are likely to be most redundant?
 - Which feature is likely to be least useful for predicting price?

In [ ]:
# YOUR CODE HERE

---
### Step 4: Modeling

### <span style="color:chocolate">Exercise 9:</span> Baseline model (5 points)

Let's start by evaluating a baseline model. Precisely, you'll use the average price of cars in the training set as our baseline model -- that is, the baseline always predicts the average price regardless of the input.

1. Implement this baseline using the Y_train_std data and print the average price. Note: You can revert the price variable to the original scale for interpretation purposes.

In [ ]:
# YOUR CODE HERE

### <span style="color:chocolate">Exercise 10:</span> Improvement over Baseline with TensorFlow (10 points)

Let's train a linear regression model much like we did in the previous assignment, but this time using TensorFlow. 

1. Fill in the <span style="color:green">NotImplemented</span> parts of the build_model() function below by following the instructions provided as comments. Hint: refer to Demo 3 in [bCourses/Modules/Live Session Demos](https://bcourses.berkeley.edu/courses/1534588/files/88733489?module_item_id=17073646) for an example.
2. Build and compile a model using the build_model() function and the (X_train_std, Y_train_std) data. Set learning_rate = 0.0001. Call the resulting object *model_tf*.
3. Train *model_tf* using the (X_train_std, Y_train_std) data. Set num_epochs = 5. Pass the (X_val_std, Y_val_std) data for validation. Hint: see the documentation behind the [tf.keras.Model.fit()](https://bcourses.berkeley.edu/courses/1534588/files/88733489?module_item_id=17073646) method.
3. Generate a plot with the loss values on the y-axis and the epoch number on the x-axis for visualization. Make sure to include axes name and title. Hint: check what the [tf.keras.Model.fit()](https://bcourses.berkeley.edu/courses/1534588/files/88733489?module_item_id=17073646) method returns.

More notes on point 1: the idea is to build a *computational graph* for linear regression, and then send data through it. There are many ways to build graphs, but [TenforFlow Keras API](https://www.tensorflow.org/api_docs/python/tf/keras) is recommended. 

In [ ]:
def build_model(num_features, learning_rate):
  """Build a TF linear regression model using Keras.

  Args:
    num_features: The number of input features.
    learning_rate: The desired learning rate for SGD.

  Returns:
    model: A tf.keras model (graph).
  """
  # This is not strictly necessary, but each time you build a model, TF adds
  # new nodes (rather than overwriting), so the colab session can end up
  # storing lots of copies of the graph when you only care about the most
  # recent. Also, as there is some randomness built into training with SGD,
  # setting a random seed ensures that results are the same on each identical
  # training run.
  tf.keras.backend.clear_session()
  tf.random.set_seed(0)

  # Build a model using keras.Sequential. While this is intended for neural
  # networks (which may have multiple layers), we want just a single layer for
  # linear regression.
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Dense(
      units=NotImplemented,        # output dim
      input_shape=NotImplemented,  # input dim
      use_bias=True,               # use a bias (intercept) param
      kernel_initializer=NotImplemented,  # initialize params to 1
      bias_initializer=NotImplemented,    # initialize bias to 1
  ))

  # We need to choose an optimizer. We'll use GD, which is actually mini-batch GD
  optimizer = NotImplemented

  # Finally, compile the model. This finalizes the graph for training.
  # We specify the MSE loss and the optimizer above
  NotImplemented
    
  return model

In [ ]:
tf.random.set_seed(0)
# 2. Build and compile model
# YOUR CODE HERE

# 3. Fit the model
# YOUR CODE HERE

---
### Step 5: Hyperparameter tuning

Hyperparameter tuning is a crucial step in optimizing ML models. It involves systematically adjusting hyperparameters such as learning rate, number of epochs, and optimizer to find the model configuration that leads to the best generalization performance.

This tuning process is typically conducted by monitoring the model's performance on the validation vs. training set. It's important to note that using the test set for hyperparameter tuning can compromise the integrity of the evaluation process by violating the assumption of "blindness" of the test data.

### <span style="color:chocolate">Exercise 11:</span> Hyperparameter tuning (10 points)

1. Fine-tune the **learning rate** and **number of epochs** hyperparameters of *model_tf* to determine the setup that yields the most optimal generalization performance. Feel free to explore various values for these hyperparameters. Hint: you can manually test different hyperparameter values or you can use the [Keras Tuner](https://www.tensorflow.org/tutorials/keras/keras_tuner). If you decide to work with the Keras Tuner, define a new model building function named <span style="color:chocolate">build_model_tuner()</span>.

After identifying your preferred model configuration, print the following information:

2. The learned parameters of the tuned model (this should include the bias term). Hint: use  <span style="color:chocolate">[model_name].layers[0].get_weights()</span>.
3. The loss at the final epoch on both the training and validation datasets;
4. The difference between the last-epoch loss observed on the training and validation datasets.


Please note that we will consider 'optimal model configuration' any last-epoch training loss that is below 0.31 and any last epoch validation loss that is below 0.48.

In [ ]:
tf.random.set_seed(0)
# YOUR CODE HERE

---
### Step 6: Evaluation and Generalization


Now that you've determined the optimal set of hyperparameters, it's time to evaluate your optimized (tuned) model on the test data to gauge its performance in real-world scenarios, commonly known as inference.

### <span style="color:chocolate">Exercise 12:</span> Computing MSE (10 points)

1. Calculate the MSE on both (X_train_std, Y_train_std) and (X_test_std, Y_test_std) datasets. Hint: You can utilize the <span style="color:chocolate">model.evaluate()</span> method provided by tf.keras.

2. Does the model demonstrate strong generalization capabilities? Provide an explanation based on your observations.

4. Generate a plot to visualize the accuracy of the predictions. Plot the actual (observed) Y_test values on the x-axis and the predicted Y_test values on the y-axis. Additionally, include a 45-degree line in the plot for reference. Ensure that the plot contains appropriate axis labels and a title. Provide commentary on the model's fit based on this visualization. Hint: You can utilize the <span style="color:chocolate">model.predict()</span> method available in tf.keras. 

In [ ]:
# YOUR CODE HERE

----
#### <span style="color:chocolate">Additional practice question</span> (not graded)

In Exercise 12, you reported an aggregated MSE. Let's revisit the exercise by:

1. Performing a subgroup evaluation of the model. Specifically, calculate the test data MSE for the following makes: ['alfa-romero', 'audi', 'chevrolet', 'dodge', 'honda'].
2. Addressing the question: Is the model "fair" across each make?

In [ ]:
# YOUR CODE HERE